In [ ]:
from scipy import signal
# import statsmodels.formula.api as sm
from sklearn import linear_model
from daq.pico import Data
import holoviews as hv
import datashader as ds
import numpy as np
from holoviews.operation.datashader import datashade, dynspread
hv.extension('bokeh')


In [ ]:
%opts RGB [width=800, height=350]

In [ ]:
data = Data()
dfd = data.load('./data/20171018-0012.nc')


In [ ]:
df = dfd.copy()

b, a = signal.butter(1, 0.0001)

df.loc[:, 'res_volt_smooth'] = signal.filtfilt(b, a, df.res_volt.values)
df.loc[:, 'sec_volt_smooth'] = signal.filtfilt(b, a, df.sec_volt.values)
df.loc[:, 'sin'] = np.sin(2 * np.pi * 20 * df.t)
df.loc[:, 'cos'] = np.cos(2 * np.pi * 20 * df.t)

model = linear_model.LinearRegression()
X = df.loc[:, ['sin', 'cos']].values
y = np.expand_dims(df.res_volt.values, 1)
model.fit(X, y)
df.loc[:, 'res_harmonic_fit'] = model.predict(X)[:, 0]

model = linear_model.LinearRegression()
X = df.loc[:, ['sin', 'cos']].values 
y = np.expand_dims(df.sec_volt.values, 1)
model.fit(X, y)
df.loc[:, 'sec_harmonic_fit'] = model.predict(X)[:, 0]


df.loc[:, 'res_resid'] = df.res_volt - df.res_harmonic_fit
df.loc[:, 'sec_resid'] = df.sec_volt - df.sec_harmonic_fit



df.head()


In [ ]:
c1 = hv.Points((df.sec_harmonic_fit, df.sec_resid), kdims=['Harmonic Fit', 'Fit Residual'])
c1 = datashade(c1, aggregator=ds.reductions.count(), cmap=pl.get_cmap('inferno'))


dynspread(c1)

In [ ]:
c1 = hv.Points((df.res_harmonic_fit, df.res_resid), kdims=['Harmonic Fit', 'Fit Residual'])
c1 = datashade(c1, aggregator=ds.reductions.count(), cmap=pl.get_cmap('inferno'))
c1

In [ ]:
import pylab as pl
pl.get_cmap('viridis')

In [ ]:
c1 = hv.Curve((df.t, df.res_volt))
c1 = datashade(c1, aggregator=ds.reductions.any(), cmap=['blue'])

c2 = hv.Curve((df.t, df.res_volt_smooth))
c2 = datashade(c2, aggregator=ds.reductions.any(), cmap=['red'])

c3 = hv.Curve((df.t, df.harmonic_fit))
c3 = datashade(c3, aggregator=ds.reductions.any(), cmap=['green'])

c1 * c2 * c3

In [ ]:
c1 = hv.Curve((df.t, df.sec_volt))
c1 = datashade(c1, aggregator=ds.reductions.any(), cmap=['blue'])

c2 = hv.Curve((df.t, df.sec_volt_smooth))
c2 = datashade(c2, aggregator=ds.reductions.any(), cmap=['red'])

c1 * c2

In [ ]:
data.meta

In [ ]:
.001*(.5 / 2.3900000001297883e-06)